# Probing part 1

Vantage points will probe either the targets themselves (step 2) or other addresses in the same /24 prefix (step 2).

Vantage points are only the anchors.  
As always, targets are the anchors.  

This notebook is an implementation of the million scale method. Check the paper for more information.  
To do after create_datasets.ipynb

In [2]:
import uuid

from logger import logger
from scripts.utils.file_utils import load_json
from scripts.utils.measurement_utils import (
    load_targets, 
    load_vps,
    get_measurement_config,
    save_measurement_config,
    get_target_prefixes,
    ping_prefixes,
    ping_targets,
    retrieve_results,
    get_latest_measurements,
    insert_prefix_results,
    insert_target_results,
)
from default import (
    USER_ANCHORS_FILE, 
    USER_HITLIST_FILE, 
    PREFIX_MEASUREMENT_RESULTS,
    TARGET_MEASUREMENT_RESULTS,
    MEASUREMENT_CONFIG_PATH,
)

# will define the number of vps and targets to use
NB_TARGETS = 2
NB_VPS = 4

## Load targets and vps dataset

In [2]:

targets = load_targets(USER_ANCHORS_FILE, nb_target=NB_TARGETS)
vps = load_vps(USER_ANCHORS_FILE, nb_vps=NB_VPS)

# all target prefixes
target_prefixes = get_target_prefixes(targets)

# responsive IP addresses in all /24 prefixes
targets_per_prefix = load_json(USER_HITLIST_FILE)

logger.info(f"nb targets: {len(targets)}")
logger.info(f"nb_vps : {len(vps)}")

2023-09-13 15:35:46::INFO:root:2498612375:: nb targets: 2
2023-09-13 15:35:46::INFO:root:2498612375:: nb_vps : 4


## Generate measurement config

This configuration is used to retrieve all measurements results from RIPE Atlas using their API.

In [3]:
# measurement configuration for retrieval
experiment_uuid = uuid.uuid4()
target_measurement_uuid = uuid.uuid4()
prefix_measurement_uuid = uuid.uuid4()

measurement_config = get_measurement_config(
    experiment_uuid=experiment_uuid,
    target_measurement_uuid=target_measurement_uuid,
    prefix_measurement_uuid=prefix_measurement_uuid,
    targets=targets, 
    target_prefixes=target_prefixes,
    vps=vps,
)

save_measurement_config(measurement_config, MEASUREMENT_CONFIG_PATH)

# Step 1: probing each target prefixes

## Probe target prefixes
WARNING : Time consumming section

In [4]:
ping_prefixes(
    measurement_uuid=prefix_measurement_uuid,
    measurement_config=measurement_config,
    target_prefixes=target_prefixes,
    targets_per_prefix=targets_per_prefix,
    vps=vps,
)

save_measurement_config(measurement_config, MEASUREMENT_CONFIG_PATH)

2023-09-13 15:35:46::INFO:root:measurement_utils:: No cached results available
2023-09-13 15:35:46::INFO:root:measurement_utils:: Starting measurements a5a349de-8aac-42d5-b636-877fa35c7a30 with parameters: dry_run=False; nb_targets=5803825; nb_vps=4.


vps list [7257, 7146, 6719, 6934]
vps list [7257, 7146, 6719, 6934]
vps list [7257, 7146, 6719, 6934]
vps list [7257, 7146, 6719, 6934]
vps list [7257, 7146, 6719, 6934]
vps list [7257, 7146, 6719, 6934]


2023-09-13 15:35:49::INFO:root:ping_and_traceroute_classes:: measurement : a5a349de-8aac-42d5-b636-877fa35c7a30 done


# Step 2: probing each target

## Probe targets
WARNING : Time consumming section

In [5]:
# measurement configuration for retrieval
ping_targets(
    measurement_uuid=target_measurement_uuid,
    measurement_config=measurement_config,
    targets=targets,
    vps=vps,
)
save_measurement_config(measurement_config, MEASUREMENT_CONFIG_PATH)


2023-09-13 15:35:49::INFO:root:measurement_utils:: No cached results available
2023-09-13 15:35:49::INFO:root:measurement_utils:: Starting measurements 8a0e5848-9045-42dd-b889-e1cde82364d2 with parameters: dry_run=False; nb_targets=2; nb_vps=4.
2023-09-13 15:35:50::INFO:root:ping_and_traceroute_classes:: measurement : 8a0e5848-9045-42dd-b889-e1cde82364d2 done


## Retrieve prefix results
WARNING : Time consuming section

Note: it might take some time before measurement results are available through RIPE API. If no results are available, retry after a few minutes (or hours, it might really depends on the probe itself).

In [4]:
measurement_config = get_latest_measurements(MEASUREMENT_CONFIG_PATH)
logger.info(measurement_config)

prefix_measurement_uuid = measurement_config["prefix_measurements"]["measurement_uuid"]

response = retrieve_results(prefix_measurement_uuid,PREFIX_MEASUREMENT_RESULTS, NB_TARGETS, NB_VPS)

insert_prefix_results(response)

2023-09-13 15:41:04::INFO:root:3310127158:: {'experiment_uuid': 'e1fe7606-0c6a-4c58-8d46-4599730b8ccf', 'status': 'finished', 'start_time': '2023-09-13 15:35:46.679588', 'end_time': '2023-09-13 15:35:50.511055', 'is_dry_run': False, 'nb_targets': 2, 'nb_vps': 4, 'description': 'measurements from a set of vps towards all targets/target prefixes', 'af': 4, 'target_measurements': {'measurement_uuid': '8a0e5848-9045-42dd-b889-e1cde82364d2', 'targets': ['27.131.108.126', '195.66.241.220'], 'vps': {'193.31.31.236': {'id': 7257, 'address_v4': '193.31.31.236', 'asn_v4': 210715, 'country_code': 'GB', 'geometry': {'type': 'Point', 'coordinates': [-0.0205, 51.4905]}}, '85.118.58.213': {'id': 7146, 'address_v4': '85.118.58.213', 'asn_v4': 29075, 'country_code': 'FR', 'geometry': {'type': 'Point', 'coordinates': [2.3795, 48.7805]}}, '54.39.185.31': {'id': 6719, 'address_v4': '54.39.185.31', 'asn_v4': 16276, 'country_code': 'CA', 'geometry': {'type': 'Point', 'coordinates': [-73.8925, 45.3115]}}, '2

## Retrieve traget results
WARNING : Time consumming section

Note: it might take some time before measurement results are available through RIPE API. If no results are available, retry after a few minutes (or hours, it might really depends on the probe itself).

In [5]:
measurement_config = get_latest_measurements(MEASUREMENT_CONFIG_PATH)

target_measurement_uuid = measurement_config["target_measurements"]["measurement_uuid"]

response = retrieve_results(target_measurement_uuid, TARGET_MEASUREMENT_RESULTS, NB_TARGETS, NB_VPS)

insert_target_results(response)

2023-09-13 15:41:07::INFO:root:measurement_utils:: nb measurements retrieved: 8
2023-09-13 15:41:07::INFO:root:measurement_utils:: Target measurements successfully inserted in table : user_vps_to_target
